In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

class BasketballTracker:
    def __init__(self, model_path='yolov8m.pt', confidence=0.4):
        """
        Initialize the Basketball Tracker with YOLO and Hough Circle detection
        
        Args:
            model_path: Path to the YOLO model (can be pretrained or custom-trained)
            confidence: Confidence threshold for YOLO detections
        """
        self.model = YOLO(model_path)
        self.confidence = confidence
        self.ball_positions = []  # Store ball positions over time
        self.is_initialized = False  # Track whether we've found the ball in the first frame
        self.last_bbox = None  # Store the last detected bounding box
        self.roi_padding = 50  # Padding around initial detection for ROI
        
    def initialize_with_hough(self, frame):
        """Initialize ball detection using Hough Circle Transform"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, 5)
        rows = gray.shape[0]
        
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, rows / 8,
                                  param1=100, param2=40,
                                  minRadius=1, maxRadius=30)
        
        if circles is not None:
            circles = np.uint16(np.around(circles))
            
            for i in circles[0, :]:
                center = (i[0], i[1])
                # circle center
                cv2.circle(frame, center, 1, (0, 100, 100), 3)
                # circle outline
                radius = i[2]
                cv2.circle(frame, center, radius, (255, 0, 255), 3)
            
            if len(circles[0]) > 1:
                # If multiple circles, pick the strongest circle
                # You could add more sophisticated selection here
                strongest_circle = circles[0, 0]
            else:
                strongest_circle = circles[0, 0]
            
            x, y, r = strongest_circle  # Get center x, y and radius
            
            scaled_r = r * 1.2  # bounding box is best with slightly larger than exact circle
            
            # Convert to bounding box format (x, y, width, height)
            bbox = (int(x-scaled_r), int(y-scaled_r), int(2.1*scaled_r), int(2.1*scaled_r))
            self.is_initialized = True
            self.last_bbox = bbox
            
            return bbox, 1.0  # High confidence for initial detection
        print("Hough detectin returning None")
        return None, 0.0
    
    def track_with_yolo(self, frame, prev_bbox):
        """Track ball using YOLO with region of interest from previous detection"""
        x, y, w, h = prev_bbox
        
        # Create ROI with padding
        roi_x = max(0, x - self.roi_padding)
        roi_y = max(0, y - self.roi_padding)
        roi_w = min(frame.shape[1] - roi_x, w + 2 * self.roi_padding)
        roi_h = min(frame.shape[0] - roi_y, h + 2 * self.roi_padding)
        
        # Extract ROI
        roi = frame[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]
        
        # Skip if ROI is too small
        if roi.size == 0 or roi_w <= 0 or roi_h <= 0:
            return prev_bbox, 0.5
        
        # Run YOLO on ROI
        results = self.model(roi, verbose=False, conf=0.1, iou=0.45, agnostic_nms=True)
        
        basketball_detections = []

        for r in results:
            boxes = r.boxes
            for box in boxes:
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                cls_name = self.model.names[cls] if hasattr(self.model, 'names') else f"Class {cls}"
                print(f"Detected: {cls_name} (ID: {cls}) with confidence {float(box.conf[0]):.2f}")
                is_ball = (cls == 32 or cls == 54 or (hasattr(self.model, 'names') and any(ball_word in self.model.names[cls].lower() for ball_word in ['ball', 'basketball', 'sphere', 'donut'])))
                
                # Check for basketball (class 32 in COCO)
                if is_ball and conf > self.confidence:
                    # Extract bounding box
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    # Convert coordinates back to original frame
                    x1 += roi_x
                    y1 += roi_y
                    x2 += roi_x
                    y2 += roi_y
                    
                    basketball_detections.append({
                        'bbox': (int(x1), int(y1), int(x2-x1), int(y2-y1)),
                        'confidence': conf
                    })
        
        # If YOLO detection fails, try color-based detection as fallback
        if not basketball_detections:
            print("failed to detect ball")
            bbox = self.last_bbox
            if bbox is not None:
                basketball_detections.append({
                    'bbox': bbox,
                    'confidence': 1
                })
        
        # Return the highest confidence detection or previous bbox if none found
        if basketball_detections:
            best_detection = max(basketball_detections, key=lambda x: x['confidence'])
            return best_detection['bbox'], best_detection['confidence']
        
        return prev_bbox, 0.3  # Return previous bbox with lower confidence
    
    
    def track_in_video(self, video_source=0, visualize=True):
        """Track basketball in video using Hough for initialization and YOLO for tracking"""
        cap = cv2.VideoCapture(video_source)
        self.ball_positions = []
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Initial detection with Hough Circle Transform
            if not self.is_initialized:
                bbox, confidence = self.initialize_with_hough(frame)
                if bbox is None:
                    # Continue to next frame if no initial detection
                    print("No initial detection, moving to next frame")
                    if visualize:
                        cv2.imshow('Basketball Tracking', frame)
                        if cv2.waitKey(1) & 0xFF == ord('q'):
                            break
                    continue
            else:
                # Continue tracking with YOLO
                bbox, confidence = self.track_with_yolo(frame, self.last_bbox)
                self.last_bbox = bbox
            
            # Process detection
            if bbox:
                x, y, w, h = bbox
                # Calculate center of the ball
                center_x = x + w // 2
                center_y = y + h // 2
                
                # Store position
                self.ball_positions.append((center_x, center_y))
                
                # Visualize
                if visualize:
                    # Draw bounding box
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    # Draw center point
                    cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)
                    # Show confidence
                    cv2.putText(frame, f"Ball: {confidence:.2f}", (x, y - 10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    # Draw trajectory
                    if len(self.ball_positions) > 1:
                        for i in range(1, len(self.ball_positions)):
                            cv2.line(frame, self.ball_positions[i-1], self.ball_positions[i], 
                                   (255, 0, 0), 2)
            
            if visualize:
                cv2.imshow('Basketball Tracking', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        
        cap.release()
        if visualize:
            cv2.destroyAllWindows()
        
        return self.ball_positions
    
    def analyze_shot(self):
        """
        Analyze the shot trajectory based on tracked positions
        
        Returns:
            Dictionary with shot analytics
        """
        if len(self.ball_positions) < 10:  # Need minimum positions for analysis
            return {"success": False, "error": "Not enough tracking points"}
            
        # Extract y-coordinates (height) over time
        heights = [pos[1] for pos in self.ball_positions]
        
        # Find peak of trajectory (highest point)
        min_height_idx = heights.index(min(heights))  # Y is inverted in image coordinates
        peak_position = self.ball_positions[min_height_idx]
        
        # Calculate release angle (using first few points)
        if len(self.ball_positions) > 5:
            start_x, start_y = self.ball_positions[0]
            end_x, end_y = self.ball_positions[5]
            # Calculate angle (adjust for image coordinate system)
            dx = end_x - start_x
            dy = start_y - end_y  # Invert y to match standard coordinates
            release_angle = np.degrees(np.arctan2(dy, dx))
        else:
            release_angle = None
            
        # Determine if shot was made (this is simplified and would need to be enhanced)
        # In a real app, you'd need court detection to know basket location
        
        # Detect if trajectory goes up then down (parabolic)
        has_parabolic_path = False
        if min_height_idx > 5 and min_height_idx < len(heights) - 5:
            has_parabolic_path = True
            
        return {
            "success": True,
            "peak_position": peak_position,
            "release_angle": release_angle,
            "has_parabolic_path": has_parabolic_path,
            "tracked_points": len(self.ball_positions)
        }


# Example usage
if __name__ == "__main__":
    # Initialize tracker with default YOLO model
    tracker = BasketballTracker()
    
    # Track basketball in video (0 for webcam or provide video file path)
    video_path = "../input/video/video4.mov"  # Replace with your video file
    tracker.track_in_video(video_path)
    
    # Analyze the shot
    analysis = tracker.analyze_shot()
    print("Shot Analysis:", analysis)

Detected: person (ID: 0) with confidence 0.26
Detected: refrigerator (ID: 72) with confidence 0.15
Detected: sports ball (ID: 32) with confidence 0.11
failed to detect ball
Detected: bird (ID: 14) with confidence 0.32
Detected: refrigerator (ID: 72) with confidence 0.23
Detected: person (ID: 0) with confidence 0.17
failed to detect ball
Detected: person (ID: 0) with confidence 0.13
Detected: refrigerator (ID: 72) with confidence 0.11
Detected: cake (ID: 55) with confidence 0.10
failed to detect ball
Detected: person (ID: 0) with confidence 0.21
Detected: refrigerator (ID: 72) with confidence 0.16
Detected: cake (ID: 55) with confidence 0.15
Detected: sink (ID: 71) with confidence 0.15
failed to detect ball


2025-04-05 23:02:10.558 Python[51987:5643169] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-05 23:02:10.558 Python[51987:5643169] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Detected: person (ID: 0) with confidence 0.15
Detected: donut (ID: 54) with confidence 0.12
Detected: refrigerator (ID: 72) with confidence 0.11
failed to detect ball
Detected: refrigerator (ID: 72) with confidence 0.20
Detected: person (ID: 0) with confidence 0.14
Detected: sports ball (ID: 32) with confidence 0.13
Detected: person (ID: 0) with confidence 0.10
failed to detect ball
Detected: refrigerator (ID: 72) with confidence 0.29
Detected: sports ball (ID: 32) with confidence 0.22
Detected: person (ID: 0) with confidence 0.17
failed to detect ball
Detected: refrigerator (ID: 72) with confidence 0.25
Detected: person (ID: 0) with confidence 0.24
Detected: person (ID: 0) with confidence 0.11
failed to detect ball
Detected: person (ID: 0) with confidence 0.57
Detected: donut (ID: 54) with confidence 0.43
Detected: refrigerator (ID: 72) with confidence 0.21
Detected: person (ID: 0) with confidence 0.29
Detected: refrigerator (ID: 72) with confidence 0.21
Detected: refrigerator (ID: 72

2025-04-05 23:02:19.733 Python[51987:5643169] _TIPropertyValueIsValid called with 16 on nil context!
2025-04-05 23:02:19.733 Python[51987:5643169] imkxpc_getApplicationProperty:reply: called with incorrect property value 16, bailing.
2025-04-05 23:02:19.733 Python[51987:5643169] Text input context does not respond to _valueForTIProperty:


Detected: refrigerator (ID: 72) with confidence 0.35
Detected: refrigerator (ID: 72) with confidence 0.23
Detected: refrigerator (ID: 72) with confidence 0.11
failed to detect ball
Detected: refrigerator (ID: 72) with confidence 0.28
Detected: refrigerator (ID: 72) with confidence 0.22
Detected: refrigerator (ID: 72) with confidence 0.13
failed to detect ball
